# Run other models from tabsyn repo

* tabddpm
* stasy
* codi
* great

**NOTE:** You have to run RunTabSyn.ipynb up to Prepare-data-for-model section, to make this notebook work

## TabDDPM

36 min for training with 10000 epochs and 90 min for sampling

In [1]:
from tabsyn_lib.src.util import load_config, dump_config

In [7]:
# it is logging too much, let's turn it down
!sed -i 's/self.log_every = 1/self.log_every = 50/' tabsyn_lib/baselines/tabddpm/train.py

In [10]:
# fix script bug
!sed -i 's/save_path = args.save_path/save_path = sample_save_path + "\/sampled.csv"/' tabsyn_lib/baselines/tabddpm/sample.py

In [2]:
conf = load_config('tabsyn_lib/baselines/tabddpm/configs/adult.toml')

In [3]:
conf['parent_dir'] = 'tabsyn_lib/baselines/tabddpm/configs/act-mooc'
conf['model_save_path'] = 'tabsyn_lib/baselines/tabddpm/ckpt/act-mooc'
conf['sample_save_path'] = 'tabsyn_lib/baselines/tabddpm/sampled/act-mooc'
conf['real_data_path'] = 'act-mooc-train.csv.gz'
conf['num_numerical_features'] = 5
conf['sample']['num_samples'] = 271341
conf['train']['main']['steps'] = 100000 // 10

In [4]:
dump_config(conf, 'tabsyn_lib/baselines/tabddpm/configs/act-mooc.toml')

In [5]:
conf

{'parent_dir': 'tabsyn_lib/baselines/tabddpm/configs/adult',
 'model_save_path': 'tabsyn_lib/baselines/tabddpm/ckpt/adult',
 'sample_save_path': 'tabsyn_lib/baselines/tabddpm/sampled/adult',
 'real_data_path': 'act-mooc-train.csv.gz',
 'num_numerical_features': 5,
 'model_type': 'mlp',
 'task_type': 'binclass',
 'model_params': {'num_classes': 2,
  'is_y_cond': False,
  'rtdl_params': {'d_layers': [1024, 2048, 2048, 1024], 'dropout': 0.0}},
 'diffusion_params': {'num_timesteps': 1000, 'gaussian_loss_type': 'mse'},
 'train': {'main': {'steps': 10000,
   'lr': 0.001809824563637657,
   'weight_decay': 0.0005,
   'batch_size': 4096},
  'T': {'seed': 0,
   'normalization': 'quantile',
   'num_nan_policy': 'mean',
   'cat_nan_policy': None,
   'cat_min_frequency': None,
   'cat_encoding': None,
   'y_policy': 'default'}},
 'sample': {'num_samples': 271341, 'batch_size': 10000, 'seed': 0},
 'eval': {'type': {'eval_model': 'mlp', 'eval_type': 'synthetic'},
  'T': {'seed': 0,
   'normalization'

In [ ]:
%time !python tabsyn_lib/main.py --dataname act-mooc --method tabddpm --mode train

In [ ]:
!python tabsyn_lib/main.py --dataname act-mooc --method tabddpm --mode sample  # --save_path [PATH_TO_SAVE]

## STaSy

In [15]:
# sklearn has changed API in version 1.2
!sed -i 's/sparse=False/sparse_output=False/' tabsyn_lib/src/data.py

In [6]:
# get rid of parallel processing due to errors with shm
!sed -i 's/num_workers=4/num_workers=0/' tabsyn_lib/baselines/stasy/main.py

In [ ]:
# Prevent ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
%time !python tabsyn_lib/main.py --dataname act-mooc --method stasy --mode train

In [ ]:
!python main.py --dataname act-mooc --method stasy --mode sample  # --save_path [PATH_TO_SAVE]

## CoDi

In [ ]:
# 90s / iteration, 60 iterations per epoch
%time !python tabsyn_lib/main.py --dataname act-mooc --method codi --mode train --total_epochs_both 20

In [ ]:
!python main.py --dataname act-mooc --method codi --mode sample  # --save_path [PATH_TO_SAVE]

## GReaT

14 hours for 25 epochs (default 100 in baselines/great/main.py:25)

In [ ]:
%time !python tabsyn_lib/main.py --dataname act-mooc --method great --mode train

In [ ]:
!python main.py --dataname act-mooc --method great --mode sample  # --save_path [PATH_TO_SAVE]